## **Lectura 2: Formulación implícita**

### Modelando en Gurobi: Desde lo Matemático a Python (Nivel Avanzado)

**1. Conjuntos**

*Expresión Matemática:* 
- $ I = \{1, 2, ..., n\} $

- $ J = \{a, b, ..., z\} $

- $ K = \{(i, j) | i \in I, j \in J \} $ (Producto cruzado)


*En Python:*
```python
I = range(1, n+1)
J = list(string.ascii_lowercase)
K = [(i, j) for i in I for j in J]
```


**2. Parámetros**

Pueden ser determinísticos o estocásticos, unidimensionales o multidimensionales.

*Expresión Matemática:* 

- $ c_{ij} $ (costos asociados al conjunto $K $, matriz)
- $ d_{ijk} $ (demanda tridimensional para conjuntos $I, J, K $)



*En Python:*
```python
c = {(i, j): random.uniform(1, 10) for i, j in K}  # Un diccionario con costos aleatorios
d = {(i, j, k): random.randint(5, 20) for i in I for j in J for k in K}
```



**3. Variables**

Consideremos variables binarias, continuas, y enteras, todas multidimensionales.

*Expresión Matemática:* 
$ x_{ijk} $ (cantidad del producto $i $ a enviar al destino $j $ en el periodo $k $)
$ y_{ij} \in \{0, 1\} $ (variable binaria que es 1 si se realiza el envío $i $ a $j $)
$ z_{i} $ (variable entera que representa un cierto volumen asociado con $i $)


*En Python:*
```python
x = m.addVars(I, J, K, vtype=GRB.CONTINUOUS, name="x")
y = m.addVars(I, J, vtype=GRB.BINARY, name="y")
z = m.addVars(I, vtype=GRB.INTEGER, name="z")
```



**4. Restricciones**

Supongamos que, además de la demanda, tenemos restricciones de capacidad y lógicas.

*Expresión Matemática:* 

a) Demanda:
$ \sum_{i \in I} x_{ijk} \leq d_{ijk} \quad \forall j \in J, k \in K $



```python
M = 1000
T = 50
# a) Demanda
for j in J:
    for k in K:
        m.addConstr(quicksum(x[i, j, k] for i in I) <= d[i, j, k], name=f"demanda_{j}_{k}")
```

b) Capacidad:
$ \sum_{j \in J} x_{ijk} \leq z_i \times M \quad \forall i \in I, k \in K $ (donde $M $ es una constante grande)


    
```python
M = 1000
# b) Capacidad
for i in I:
    for k in K:
        m.addConstr(quicksum(x[i, j, k] for j in J) <= z[i] * M, name=f"capacidad_{i}_{k}")
```
    


c) Lógica (Si $y_{ij} $ es 1, entonces $\sum_k x_{ijk} $ debe ser mayor que un umbral $T $):
$ y_{ij} \times T \leq \sum_k x_{ijk} $



```python
M = 1000
T = 50
# c) Lógica
for i in I:
    for j in J:
        m.addConstr(y[i, j] * T <= quicksum(x[i, j, k] for k in K), name=f"logica_{i}_{j}")
```



**5. Función Objetivo**

Supongamos que queremos minimizar el costo total y penalizar el uso de ciertas combinaciones.

*Expresión Matemática:* 
$ \min \sum_{i \in I} \sum_{j \in J} \sum_{k \in K} c_{ij} \times x_{ijk} + P \times y_{ij} $
(donde $P $ es una penalización por usar la combinación $i, j $)



*En Python:*
```python
P = 10
m.setObjective(quicksum(c[i, j] * x[i, j, k] for i in I for j in J for k in K) + P * quicksum(y[i, j] for i in I for j in J), GRB.MINIMIZE)
```



## Ejemplo ilustrativo


Considere una empresa de consultoría que tiene tres puestos vacantes: Probador, Desarrollador Java y Arquitecto. Los tres principales candidatos (recursos) para los puestos son: Carlos, Joe y Monika. La consultora ha realizado pruebas de competencia a cada uno de los candidatos para evaluar su capacidad para desempeñar cada uno de los puestos. Los resultados de estas pruebas se denominan *puntuaciones de emparejamiento*. Supongamos que sólo se puede asignar un candidato a un puesto de trabajo y que, como máximo, se puede asignar un puesto de trabajo a un candidato.

El problema consiste en determinar una asignación de recursos y trabajos tal que cada trabajo se cumpla, cada recurso se asigne a un trabajo como máximo y la puntuación total de las asignaciones sea máxima.





El siguiente código Python importa la librería Gurobi invocable e importa la clase ``GRB`` en el espacio de nombres principal.

In [14]:
import gurobipy as gp
from gurobipy import GRB

## Problema de asignación de recursos
### Datos
La lista $R$ contiene los nombres de los tres recursos: Carlos, Joe y Monika.

La lista $J$ contiene los nombres de los puestos de trabajo: Probador, Desarrollador Java y Arquitecto.

$r \in R$: índice y conjunto de recursos. El recurso $r$ pertenece al conjunto de recursos $R$.

$j \in J$: índice y conjunto de puestos de trabajo. El trabajo $j$ pertenece al conjunto de trabajos $J$.

In [ ]:
# Resource and job sets
R = ['Carlos', 'Joe', 'Monika']
J = ['Tester', 'JavaDeveloper', 'Architect']

La capacidad de cada recurso para realizar cada uno de los trabajos se indica en la siguiente tabla de puntuaciones de concordancia:

![scores](https://github.com/Gurobi/modeling-examples/blob/master/intro_to_modeling/matching_score_data.PNG?raw=1)

Para cada recurso $r$ y trabajo $j$, existe una puntuación de coincidencia correspondiente $s$. La puntuación de concordancia $s$ sólo puede tomar valores entre 0 y 100. Es decir, $s_{r,j}$ es la puntuación de concordancia. Es decir,  $s_{r,j} \in [0, 100]$  para todos los recursos $r \in R$ y trabajos $j \in J$.



In [15]:
# Combinaciones posibles (producto cruzado)
K = [(r, j) for r in R for j in J]

# 2. Parámetros
# Datos de puntuación de coincidencia para cada combinación posible de persona-trabajo
scores = {
    ('Carlos', 'Tester'): 53,
    ('Carlos', 'JavaDeveloper'): 27,
    ('Carlos', 'Architect'): 13,
    ('Joe', 'Tester'): 80,
    ('Joe', 'JavaDeveloper'): 47,
    ('Joe', 'Architect'): 67,
    ('Monika', 'Tester'): 53,
    ('Monika', 'JavaDeveloper'): 73,
    ('Monika', 'Architect'): 47
}

El siguiente constructor crea un objeto ``Model`` vacío "m". Especificamos el nombre del modelo pasando la cadena "RAP" como argumento. El objeto ``Model`` "m" contiene un único problema de optimización. Consiste en un conjunto de variables, un conjunto de restricciones, y la función objetivo.

In [16]:
# Declare and initialize model
m = gp.Model('RAP')

## Variables de decisión

Para resolver este problema de asignación, necesitamos identificar qué recurso se asigna a qué trabajo. Introducimos una variable de decisión para cada posible asignación de recursos a trabajos. Por lo tanto, tenemos 9 variables de decisión.

Para simplificar la notación matemática de la formulación del modelo, definimos los siguientes índices para los recursos y los empleos:

![variables](https://github.com/Gurobi/modeling-examples/blob/master/intro_to_modeling/decision_variables.PNG?raw=1)

Por ejemplo, $x_{2,1}$ es la variable de decisión asociada con la asignación del recurso Joe al trabajo Tester. Por lo tanto, la variable de decisión $x_{r,j}$ es igual a 1 si el recurso $r \in R$ se asigna al trabajo $j \in J$, y 0 en caso contrario.

El método ``Model.addVars()`` crea las variables de decisión para un objeto ``Model``.
Este método devuelve un objeto ``tupledict`` de Gurobi que contiene las variables recién creadas. Proporcionamos el objeto ``K`` como primer argumento para especificar los índices de las variables. La palabra clave ``name`` se utiliza para especificar un nombre para las variables de decisión recién creadas. Por defecto, se supone que las variables no son negativas.

In [17]:
# 3. Variables
# Variables de decisión: x[r, j] será 1 si el recurso 'r' es asignado al trabajo 'j', y 0 de lo contrario.
x = m.addVars(K, vtype=GRB.BINARY, name="assign")

## Restricciones de los trabajos

Ahora hablaremos de las restricciones asociadas a los trabajos. Estas restricciones deben garantizar que cada puesto sea ocupado exactamente por un recurso.

La restricción del puesto de trabajo Probador requiere que el recurso 1 (Carlos), el recurso 2 (Joe) o el recurso 3 (Monika) estén asignados a este puesto de trabajo. Esto corresponde a la siguiente restricción.

Restricción (Probador=1)

$$
x_{1,1} + x_{2,1} + x_{3,1} = 1
$$

Del mismo modo, las restricciones para los puestos de Desarrollador Java y Arquitecto pueden definirse del siguiente modo.

Restricción (Desarrollador Java = 2)

$$
x_{1,2} + x_{2,2} + x_{3,2} = 1
$$

Restricción (Arquitecto = 3)

$$
x_{1,3} + x_{2,3} + x_{3,3} = 1
$$

Las restricciones de los trabajos están definidas por las columnas de la siguiente tabla.

![trabajos](../Images/jobs_constraints.png)

En general, la restricción para el trabajo Tester se puede definir de la siguiente manera.

$$
x_{1,1} + x_{2,1} + x_{3,1} = \sum_{r=1}^{3 } x_{r,1} = \sum_{r \in R} x_{r,1} = 1
$$

Todas las restricciones de trabajo pueden definirse de forma igualmente sucinta. Para cada trabajo $j \in J$, se toma el sumatorio de las variables de decisión sobre todos los recursos. Podemos escribir la restricción de trabajo correspondiente de la siguiente manera.

$$
\sum_{r_en R} x_{r,j} = 1
$$

El método ``Model.addConstrs()`` de la API de Gurobi/Python define las restricciones de trabajo del objeto ``Model`` "m". Este método devuelve un objeto ``tupledict`` de Gurobi que contiene las restricciones de trabajo.
El primer argumento de este método, "gp.quicksum(x[r, j] for r in R)", es el método de la suma y define el LHS de las restricciones de los trabajos de la siguiente manera:
Para cada trabajo $j$ en el conjunto de trabajos $J$, tome la suma de las variables de decisión sobre todos los recursos. El $==$ define una restricción de igualdad, y el número "1" es el RHS de las restricciones.

Estas restricciones dicen que se debe asignar exactamente un recurso a cada trabajo. El segundo argumento es el nombre de este tipo de restricciones.


In [18]:
# Restricciones de trabajo: Asegurar que cada trabajo es asignado exactamente a una persona
for j in J:
    m.addConstr(gp.quicksum(x[r, j] for r in R) == 1, name=f"job_{j}")


## Restricciones de los recursos

Las restricciones de los recursos deben garantizar que se asigne como máximo un trabajo a cada recurso. Es decir, es posible que no todos los recursos estén asignados.

Por ejemplo, queremos una restricción que requiera que Carlos esté asignado como máximo a uno de los trabajos: o bien al trabajo 1 (Probador), o bien al trabajo 2 (Desarrollador Java ), o bien al trabajo 3 (Arquitecto). Podemos escribir esta restricción de la siguiente manera.

Restricción (Carlos=1)

$$
x_{1, 1} + x_{1, 2} + x_{1, 3} \leq 1.
$$

Esta restricción es menor o igual que 1 para permitir la posibilidad de que Carlos no esté asignado a ningún trabajo. Del mismo modo, las restricciones para los recursos Joe y Monika pueden definirse del siguiente modo:

Restricción (Joe=2)

$$
x_{2, 1} + x_{2, 2} + x_{2, 3} \leq 1.
$$

Restricción (Monika=3)

$$
x_{3, 1} + x_{3, 2} + x_{3, 3} \leq 1.
$$

Obsérvese que las restricciones de recursos están definidas por las filas de la tabla siguiente.

![recursos](https://github.com/Gurobi/modeling-examples/blob/master/intro_to_modeling/resource_constraints.PNG?raw=1)

La restricción del recurso Carlos se puede definir de la siguiente manera.

$$
x_{1, 1} + x_{1, 2} + x_{1, 3} = \sum_{j=1}^{3 } x_{1,j} = \sum_{j \in J} x_{1,j} \leq 1.
$$

De nuevo, cada una de estas restricciones puede escribirse de forma sucinta. Para cada recurso $r \in R$, tomar la suma de las variables de decisión sobre todos los puestos de trabajo. Podemos escribir la restricción de recursos correspondiente de la siguiente manera.

$$
\sum_{j \in J} x_{r,j} \leq  1.
$$


In [19]:
# Restricciones de recurso: Asegurar que cada persona es asignada a lo sumo a un trabajo
for r in R:
    m.addConstr(gp.quicksum(x[r, j] for j in J) <= 1, name=f"resource_{r}")


## Función objetivo

La función objetivo es maximizar la puntuación total de las asignaciones que satisfacen las restricciones del trabajo y de los recursos.

Para la tarea Comprobador, la puntuación de correspondencia es $53x_{1,1}$, si se asigna el recurso Carlos, o $80x_{2,1}$, si se asigna el recurso Joe, o $53x_{3,1}$, si se asigna el recurso Monika.
En consecuencia, la puntuación de coincidencia para el trabajo de Comprobador es la siguiente, donde sólo un término de esta suma será distinto de cero.

$$
53x_{1,1} + 80x_{2,1} + 53x_{3,1}.
$$

Del mismo modo, las puntuaciones para los puestos de Desarrollador Java y Arquitecto se definen del siguiente modo. La puntuación para el puesto de Desarrollador Java es:

$$
27x_{1, 2} + 47x_{2, 2} + 73x_{3, 2}.
$$

La puntuación para el puesto de Arquitecto es:

$$
13x_{1, 3} + 67x_{2, 3} + 47x_{3, 3}.
$$

La puntuación total de coincidencia es la suma de cada celda de la siguiente tabla.

![objfcn](https://github.com/Gurobi/modeling-examples/blob/master/intro_to_modeling/objective_function.PNG?raw=1)

El objetivo es maximizar la puntuación total de coincidencia de las asignaciones. Por lo tanto, la función objetivo se define de la siguiente manera.

\begin{equation}
\text{Maximize} \quad (53x_{1,1} + 80x_{2,1} + 53x_{3,1}) \; +
\end{equation}

\begin{equation}
\quad (27x_{1, 2} + 47x_{2, 2} + 73x_{3, 2}) \; +
\end{equation}

\begin{equation}
\quad (13x_{1, 3} + 67x_{2, 3} + 47x_{3, 3}).
\end{equation}

Cada término entre paréntesis en la función objetivo se puede expresar de la siguiente manera.


\begin{equation}
(53x_{1,1} + 80x_{2,1} + 53x_{3,1}) = \sum_{r \in R} s_{r,1}x_{r,1}.
\end{equation}

\begin{equation}
(27x_{1, 2} + 47x_{2, 2} + 73x_{3, 2}) = \sum_{r \in R} s_{r,2}x_{r,2}.
\end{equation}

\begin{equation}
(13x_{1, 3} + 67x_{2, 3} + 47x_{3, 3}) = \sum_{r \in R} s_{r,3}x_{r,3}.
\end{equation}

Por lo tanto, la función objetivo se puede escribir de forma concisa como:

\begin{equation}
\text{Maximize} \quad \sum_{j \in J} \sum_{r \in R} s_{r,j}x_{r,j}.
\end{equation}

El método ``Model.setObjective()`` de la API de Gurobi/Python define la función objetivo del objeto ``Model`` "m". La expresión objetivo se especifica en el primer argumento de este método.
Nótese que tanto los parámetros de puntuación de coincidencia "score" como las variables de decisión de asignación "x" se definen sobre las claves "k".

El segundo argumento, ``GRB.MAXIMIZE``, es el "sentido" de la optimización. En este caso, queremos *maximizar* el total de puntuaciones coincidentes de todas las asignaciones.

In [20]:
# 5. Función Objetivo
# Maximizar la puntuación total de coincidencia de todas las asignaciones
m.setObjective(gp.quicksum(scores[r, j] * x[r, j] for r in R for j in J), GRB.MAXIMIZE)


Utilizamos el método "write()" de la API de Gurobi/Python para escribir la formulación del modelo en un archivo llamado "RAP.lp".

In [21]:
# Guardar el modelo para inspección posterior
m.write('RAP.lp')

![RAP](https://github.com/Gurobi/modeling-examples/blob/master/intro_to_modeling/RAP_lp.PNG?raw=1)

Utilizamos el método "optimize( )" de la API de Gurobi/Python para resolver el problema que hemos definido para el objeto modelo "m".

In [22]:
# Ejecutar el optimizador
m.optimize()

Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (win64)

CPU model: 11th Gen Intel(R) Core(TM) i7-1165G7 @ 2.80GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 6 rows, 9 columns and 18 nonzeros
Model fingerprint: 0x0a338f16
Variable types: 0 continuous, 9 integer (9 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+01, 8e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Presolve time: 0.00s
Presolved: 6 rows, 9 columns, 18 nonzeros
Variable types: 0 continuous, 9 integer (9 binary)
Found heuristic solution: objective 193.0000000

Root relaxation: cutoff, 0 iterations, 0.00 seconds (0.00 work units)

Explored 1 nodes (0 simplex iterations) in 0.02 seconds (0.00 work units)
Thread count was 8 (of 8 available processors)

Solution count 1: 193 

Optimal solution found (tolerance 1.00e-04)
Best objective 1.930000000000e+02, best bound 

El método ``Model.getVars()`` de la API de Gurobi/Python
recupera una lista de todas las variables del objeto Modelo "m". El atributo de variable ``.x`` se utiliza para consultar los valores de la solución y el atributo ``.varName`` se utiliza para consultar el nombre de las variables de decisión.

In [23]:
# Visulizar la solución del problema de asignación de tareas
for v in m.getVars():
    if v.x > 1e-6:
        print(v.varName, v.x)

# Visulizar el valor de la función objetivo
print('Total matching score: ', m.objVal)

assign[Carlos,Tester] 1.0
assign[Joe,Architect] 1.0
assign[Monika,JavaDeveloper] 1.0
Total matching score:  193.0


La asignación óptima es asignar

* Carlos al puesto de Probador, con una puntuación de 53
* Joe al puesto de Arquitecto, con una puntuación de 67
* Monika al puesto de Desarrollador Java, con una puntuación de 73.

La máxima puntuación total es 193.